In [1]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import os

2022-12-15 00:46:36.959870: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 00:46:45.225932: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-15 00:46:45.226315: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-15 00:46:45.226364: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [42]:
df = pd.read_csv('CICFlowMeter-4.0/bin/data/daily/2022-12-14_Flow.csv')

In [35]:
train_df = pd.read_csv('TrainingSetFinal.csv')

In [40]:
class_list = train_df.Label.unique().tolist()

In [13]:
pd.set_option('use_inf_as_na',True)
train_df.dropna(inplace=True)

In [14]:
train_df = df.drop(columns=(['Unnamed: 0', ' Fwd Header Length.1']))

KeyError: "['Unnamed: 0', ' Fwd Header Length.1'] not found in axis"

In [22]:
label_encoder = preprocessing.LabelEncoder()

train_df['Label']= label_encoder.fit_transform(train_df['Label'])

In [43]:
df = df.drop(columns=['Flow ID','Src IP','Src Port','Timestamp','Protocol','Dst IP'])

In [44]:
df.columns.values

array(['Dst Port', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd

In [45]:
pd.set_option('use_inf_as_na',True)
df.dropna(inplace=True)

In [46]:
saved_model = keras.models.load_model("modelfixed.h5")

sc = MinMaxScaler()

x = pd.get_dummies(df.drop(['Label', ], axis = 1))
x = sc.fit_transform(x)

df = df.drop(columns=(['Label']))
fit_new_input = sc.transform(df)
# print(fit_new_input)
pred = saved_model.predict(fit_new_input)
# print(pred)
pred_class = np.argmax(pred, axis=-1)
# print(pred_class)

predict = label_encoder.inverse_transform(pred_class)
predict_list = predict.tolist()
for prediction in predict_list:
    print(class_list[prediction])

[[1.32384577e-03 5.82002537e-04 3.85604113e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.33079596e-03 5.33995161e-02 8.74035990e-03 ... 0.00000000e+00
  4.99651445e-02 4.99651445e-02]
 [8.23961608e-01 1.28083892e-05 1.28534704e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [7.33079596e-03 4.47264424e-01 3.85604113e-04 ... 0.00000000e+00
  4.48341625e-01 4.48341625e-01]
 [7.33079596e-03 4.38652870e-01 5.14138817e-04 ... 0.00000000e+00
  4.39964170e-01 4.39964170e-01]
 [8.36935297e-01 2.06834235e-05 5.14138817e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
14/14 [==============================] - 2s 12ms/step
[[1.0000000e+00 3.5506105e-24 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [7.7909116e-16 6.8568069e-16 0.0000000e+00 ... 0.0000000e+00
  2.8611880e-17 0.0000000e+00]
 ...
 [9.9840504e-01 1.5950266e-03 1.5550605e-24 ... 3.9606482e-34


In [8]:
len(df.columns.values)

77

In [9]:
df.columns.values

array(['Dst Port', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd

In [10]:
finalDf = pd.read_csv('TrainingSetFinal.csv')

In [11]:
test = [i for i in finalDf.columns.values.tolist() if i not in df.columns.values]
print(len(test))
print(test)

3
['Unnamed: 0', ' Fwd Header Length.1', 'Label']
